In [ ]:
import gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import random

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
# Define the Q-network model using PyTorch
class DQN(nn.Module):
    def __init__(self, state_shape, num_actions):
        super(DQN, self).__init__()
        self.conv1 = nn.Conv2d(4, 32, kernel_size=8, stride=4)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)
        self.fc1 = nn.Linear(self._conv2d_output(state_shape), 512)
        self.fc2 = nn.Linear(512, num_actions)

    def _conv2d_output(self, shape):
        x = torch.zeros(1, *shape)
        x = self._forward_conv(x)
        return int(np.prod(x.size()))

    def _forward_conv(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = torch.relu(self.conv3(x))
        return x.view(x.size(0), -1)

    def forward(self, x):
        x = self._forward_conv(x)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Experience replay buffer
class ReplayBuffer:
    def __init__(self, size=10000):
        self._storage = []
        self._maxsize = size

    def __len__(self):
        return len(self._storage)

    def add(self, state, action, reward, next_state, done):
        data = (state, action, reward, next_state, done)
        self._storage.append(data)
        if len(self._storage) > self._maxsize:
            self._storage.pop(0)

    def sample(self, batch_size):
        batch = random.sample(self._storage, batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)

        states = np.array(states)
        actions = np.array(actions)
        rewards = np.array(rewards)
        next_states = np.array(next_states)
        dones = np.array(dones)
        
        states = torch.tensor(states).float().to(device)
        actions = torch.tensor(actions).long().to(device)
        rewards = torch.tensor(rewards).float().to(device)
        next_states = torch.tensor(next_states).float().to(device)
        dones = torch.tensor(dones).float().to(device)

        return states, actions, rewards, next_states, dones

In [ ]:
# Initialize the environment
env = gym.make('BreakoutNoFrameskip-v4')
env = gym.wrappers.AtariPreprocessing(env, frame_skip=4, grayscale_obs=True, scale_obs=True, noop_max=30)
env = gym.wrappers.FrameStack(env, num_stack=4)

state_dim = env.observation_space.shape
num_actions = env.action_space.n

print('State shape: ', state_dim)
print('Number of actions: ', num_actions)

In [ ]:
# Instantiate the Q-network model and target model
model = DQN(state_dim, num_actions).to(device)
model_target = DQN(state_dim, num_actions).to(device)
model_target.load_state_dict(model.state_dict())

In [ ]:
# Define other parameters and hyperparameters
gamma = 0.99
epsilon = 1.0
epsilon_min = 0.1
epsilon_max = 1.0
epsilon_interval = epsilon_max - epsilon_min
batch_size = 32
max_steps_per_episode = 10000
max_memory_length = 30000
replay_buffer = ReplayBuffer(max_memory_length)
optimizer = optim.Adam(model.parameters(), lr=0.00025)
loss_function = nn.HuberLoss()

episode_reward_history = []
running_reward = 0
episode_count = 0
frame_count = 0
epsilon_random_frames = 50000
epsilon_greedy_frames = 1000000.0
update_target_network = 10000

reward_history = []
frame_count_history = []
epsilon_history = []
best_reward = 0

In [ ]:
while True:
    env.reset()
    state, _, done, _, _ = env.step(0)
    state = np.array(state, dtype=np.float32)
    episode_reward = 0

    for timestep in range(1, max_steps_per_episode):
        frame_count += 1

        if frame_count < epsilon_random_frames or epsilon > np.random.rand(1):
            action = np.random.choice(num_actions)
        else:
            with torch.no_grad():
                state = np.array(state, dtype=np.float32)
                q_values = model(torch.tensor(state).unsqueeze(0).to(device))
                action = torch.argmax(q_values).item()

        if epsilon > epsilon_min:
            epsilon -= epsilon_interval / epsilon_greedy_frames
            epsilon = max(epsilon, epsilon_min)

        next_state, reward, done, e, f = env.step(action)
        next_state =  np.array(next_state, dtype=np.float32)
        episode_reward += reward

        replay_buffer.add(state, action, reward, next_state, done)
        state = next_state

        if len(replay_buffer) > batch_size:
            states, actions, rewards, next_states, dones = replay_buffer.sample(batch_size)

            future_rewards = model_target(next_states).max(1)[0]
            updated_q_values = rewards + gamma * future_rewards * (1 - dones)

            current_q_values = model(states)
            selected_q_values = current_q_values.gather(1, actions.unsqueeze(1)).squeeze(1)

            loss = loss_function(updated_q_values, selected_q_values)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # To avoid biais de sur-estimation update the target network every 10,000 steps
        if frame_count % update_target_network == 0:
            model_target.load_state_dict(model.state_dict())

        if timestep > 5000:
            print(f"Timestep: {timestep} Done: {done} Info: {e} {f}")

        if done:
            break

    episode_reward_history.append(episode_reward)
    running_reward = np.mean(episode_reward_history)

    if episode_count % 10 == 0:
        if len(episode_reward_history) > 100:
            del episode_reward_history[:10]
        running_reward = np.mean(episode_reward_history)
        print("Episode {} reward: {} frame_count: {} epsilon: {} memory size: {}".format(episode_count, running_reward, frame_count, epsilon, len(replay_buffer)))
        reward_history.append(running_reward)
        frame_count_history.append(frame_count)
        epsilon_history.append(epsilon)

    if running_reward > 5 and running_reward > best_reward:
        best_reward = running_reward
        torch.save(model.state_dict(), f"models/model_rew_{round(running_reward)}.pth")

    if running_reward > 40:
        print("Solved at episode {}!".format(episode_count))
        break

    episode_count += 1

In [ ]:
from matplotlib import pyplot as plt

plt.figure(figsize=(20,5))
plt.plot(frame_count_history, reward_history)
plt.plot(frame_count_history, epsilon_history)
plt.title('Reward history and epsilon history over frame_count')
plt.xlabel('frame_count')
plt.legend(['reward', 'epsilon'], loc='upper left')
plt.show()

In [ ]:
torch.save(model.state_dict(), 'models/model_weights.pth')

In [ ]:
from gym.wrappers.monitoring.video_recorder import VideoRecorder
import os

def create_video(env, model, folder="videos", name="video"):
    video_recorder = VideoRecorder(env, path=os.path.join(folder, name + ".mp4"))
    env.reset()
    s, _, done, _, _ = env.step(0)
    epoch = 0

    while not done and epoch < 1000:
        video_recorder.capture_frame()
        s = np.array(s)
        action_probs = model(torch.tensor(s).unsqueeze(0).to(device))
        action = torch.argmax(action_probs).item()
        s, _, done, _, _ = env.step(action)
        epoch += 1

    video_recorder.close()

In [ ]:
env = gym.make('BreakoutNoFrameskip-v4', render_mode='rgb_array')
env = gym.wrappers.AtariPreprocessing(env, frame_skip=4, grayscale_obs=True, scale_obs=True, noop_max=30)
env = gym.wrappers.FrameStack(env, num_stack=4)
create_video(env, model, folder="videos", name="video")